In [86]:
### Project Baseline ### 
'''
1. Read in data
2. Separate out text of comments (features) and gender of employee (labels)
3. Basic feature engineering (unigrams and bigrams) 
4. Tune hyperparameters with grid search 
5. Train and evaluate Logistic Regression and SVM (linear and RBF kernels)
'''

'\n1. Read in data\n2. Separate out text of comments (features) and gender of employee (labels)\n3. Basic feature engineering (unigrams and bigrams) \n4. Tune hyperparameters with grid search \n5. Train and evaluate Logistic Regression and SVM (linear and RBF kernels)\n'

In [87]:
#Fucking unicode

# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

# DEPRECATED IN PYTHON 3 / I DON'T SEEM TO NEED UTF8

In [88]:
#Libraries we'll need 
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn import datasets, neighbors, linear_model
from sklearn.svm import SVC
import scipy.stats
from sklearn.feature_extraction import DictVectorizer
import numpy as np 
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict
import operator


In [89]:
#initially reading in data
df = pd.read_csv('articles_train_degendered_filtered.csv',encoding='utf8')

In [90]:
#selecting features and labels 
data = df[['sentence','gender']]

In [91]:
#preprocessing: eliminating punctuation
#print("Cleaning data...")
def remove_punctuations(text):
    for punctuation in string.punctuation:
        #text = str(text).encode('utf-8').replace(punctuation, ' ')
        text = text.replace(punctuation, ' ')
    #text = re.sub( '\s+', ' ', text ).encode('utf-8').strip()
    text = re.sub( '\s+', ' ', text ).strip()
    return text

In [92]:
data.loc[:, 'sentence'] = data.loc[:, 'sentence'].apply(remove_punctuations)

/anaconda3/envs/nlu/lib/python3.6/site-packages/pandas/core/indexing.py:621: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [93]:
print(data.size)

1739068


In [109]:
# returns list of top 5k unigrams and 10k bigrams
def sort_uni_bigrams():
    all_unigrams = defaultdict(int)
    all_bigrams = defaultdict(int)
    for index, row in data.iterrows():
        text = row['sentence']
        words = ['<S>'] + text.split(' ') + ['</S>']
        for i in range(1, len(words)):
            uni = words[i]
            bi = words[i-1] + " " + words[i]
            all_unigrams[uni] += 1
            all_bigrams[bi] += 1
    sorted_unigrams = sorted(all_unigrams.items(), key=operator.itemgetter(1), reverse=True)
    sorted_bigrams = sorted(all_bigrams.items(), key=operator.itemgetter(1), reverse=True)
    return (sorted_unigrams, sorted_bigrams)

In [110]:
sorted_unigrams, sorted_bigrams = sort_uni_bigrams()
top_unigrams = {uni[0]: uni[1] for uni in sorted_unigrams[:5000]}
top_bigrams = {bi[0]: bi[1] for bi in sorted_bigrams[:10000]}

In [120]:
# was getting an error b/c they were defaultdicts, so converted back to dicts
top_unigrams = dict(top_unigrams)
top_bigrams = dict(top_bigrams)

In [125]:
# given sentence, and list of top unigrams and bigrams
# returns feature dicts for unigrams and bigrams
def uni_bigrams(text):
    words = ['<S>'] + text.split(' ') + ['</S>']
    unigrams = defaultdict(int)
    bigrams = defaultdict(int)
    for i in range(1, len(words)):
        uni = words[i]
        bi = words[i-1] + " " + words[i]
        if uni in top_unigrams:
            unigrams[uni] += 1
        else:
            unigrams['UNK'] += 1
        if bi in top_bigrams:
            bigrams[bi] += 1
        else:
            bigrams['UNK UNK'] += 1
    return (unigrams, bigrams)

In [126]:
#converting samples into features 
feat_dicts = []
labels = []
for index, row in data.iterrows():
    text = row['sentence']
    label = row['gender']
    unigrams, bigrams = uni_bigrams(text)
    feature_dict = {**unigrams, **bigrams}
    feat_dicts.append(feature_dict)
    labels.append(label)

In [127]:
feat_dicts

[{'</S>': 1,
  'Beijing': 1,
  'Taiwan': 1,
  'UNK': 5,
  'UNK UNK': 22,
  'a': 1,
  'a news': 1,
  'and': 2,
  'and even': 1,
  'at': 1,
  'at a': 1,
  'back': 1,
  'back to': 1,
  'by': 1,
  'capital': 1,
  'conference': 1,
  'conference </S>': 1,
  'even': 1,
  'going': 1,
  'in': 1,
  'is': 1,
  'is going': 1,
  'it': 1,
  'it told': 1,
  'journalists': 1,
  'news': 1,
  'news conference': 1,
  'of': 1,
  'old': 1,
  'path': 1,
  'step': 1,
  'the': 2,
  'the old': 1,
  'threatening': 1,
  'to': 1,
  'to the': 1,
  'told': 1,
  '”': 1,
  '” it': 1},
 {'</S>': 1,
  '<S> Despite': 1,
  'Despite': 1,
  'Ms': 1,
  'UNK': 7,
  'UNK UNK': 19,
  'a': 1,
  'actions': 1,
  'and': 1,
  'avoid': 1,
  'had': 1,
  'it': 1,
  'it said': 1,
  'of': 1,
  'of the': 1,
  'people': 1,
  'people and': 1,
  'recent': 1,
  'relations': 1,
  'said': 1,
  'the': 2,
  'to': 1,
  'to avoid': 1,
  'vowed': 1,
  'vowed to': 1,
  'which': 1,
  'which it': 1},
 {'</S>': 1,
  '<S> “We': 1,
  'UNK': 3,
  'UNK UNK